In [1]:
import numpy as np
import time
from collections import OrderedDict

try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')

# Create a pulse sequence for the measurement

### Set desired common parameters for predefined sequence generation

In [2]:
# To change some parameters you need to use "pulsedmasterlogic.set_generation_parameters()"
# You can either set each parameter by passing it as keyword argument or give the method a dictionary containing the 
# parameters to change or a combination of both where the first argument must be the dictionary followed by a number 
# of keyword arguments.
pulsedmasterlogic.set_generation_parameters(sync_channel='', microwave_amplitude=0.25, rabi_period=200e-9)
# pulsedmasterlogic.set_generation_parameters({'sync_channel': 'd_ch4', 'microwave_amplitude': 0.25}, rabi_period=200e-9)

# Read-only access to all available and currently set global generation parameters as dictionary via property
for param, value in pulsedmasterlogic.generation_parameters.items():
    print('{0}: {1}'.format(param, value))

laser_channel: d_ch1
sync_channel: 
gate_channel: d_ch2
microwave_channel: a_ch1
microwave_frequency: 2870000000.0
microwave_amplitude: 0.25
rabi_period: 2e-07
laser_length: 3e-06
laser_delay: 5e-07
wait_time: 1e-06
analog_trigger_voltage: 0.0


### Call the desired predefined generation method with additional sequence specific parameters

In [3]:
# This property contains all available predefined generate method names together with the respective set 
# of parameters with default values.
for method, params in pulsedmasterlogic.generate_method_params.items():
    print('{0}:\n  {1}'.format(method, params))

HHamp:
  {'name': 'hh_amp', 'spinlock_length': 2e-05, 'amp_start': 0.05, 'amp_step': 0.01, 'num_of_points': 50}
HHpol:
  {'name': 'hh_pol', 'spinlock_length': 2e-05, 'spinlock_amp': 0.1, 'polarization_steps': 50}
HHtau:
  {'name': 'hh_tau', 'spinlock_amp': 0.1, 'tau_start': 1e-06, 'tau_step': 1e-06, 'num_of_points': 50}
hahnecho:
  {'name': 'hahn_echo', 'tau_start': 1e-06, 'tau_step': 1e-06, 'num_of_points': 50, 'alternating': True}
idle:
  {'name': 'idle', 'length': 3e-06}
laser_mw_on:
  {'name': 'laser_mw_on', 'length': 3e-06}
laser_on:
  {'name': 'laser_on', 'length': 3e-06}
pulsedodmr:
  {'name': 'pulsedODMR', 'freq_start': 2870000000.0, 'freq_step': 200000.0, 'num_of_points': 50}
rabi:
  {'name': 'rabi', 'tau_start': 1e-08, 'tau_step': 1e-08, 'number_of_taus': 50}
ramsey:
  {'name': 'ramsey', 'tau_start': 1e-06, 'tau_step': 1e-06, 'num_of_points': 50, 'alternating': True}
xy8_freq:
  {'name': 'xy8_freq', 'freq_start': 100000.0, 'freq_step': 10000.0, 'num_of_points': 50, 'xy8_order

In [4]:
# Get the dictionary containing the default parameters for the desired sequence and alter the values as needed.
# generate_params = pulsedmasterlogic.generate_method_params['rabi']
generate_params=OrderedDict()
generate_params['name'] = 'my_rabi'
generate_params['number_of_taus'] = 50
generate_params['tau_start'] = 10.0e-9
generate_params['tau_step'] = 10.0e-9

# Call the actual generation method using a wrapper method and pass the parameter set
pulsedmasterlogic.generate_predefined_sequence('rabi', generate_params)
# Give it a moment to generate
time.sleep(0.5)

# The created object is a PulseBlockEnsemble instance, a recipe to create a waveform.
# You can get all created PulseBlockEnsembles using this property (keys are ensemble names, values are the object instances):
for ensemble_name in pulsedmasterlogic.saved_pulse_block_ensembles.keys():
    print(ensemble_name)

my_rabi


### Sample the created PulseBlockEnsemble and write a waveform to the pulse generator (AWG)

In [5]:
# The "with_load" flag can be used to also load the waveform into the channels after creating it
pulsedmasterlogic.sample_ensemble('my_rabi', with_load=True)
# Wait for the sampling to finish. There is a status dictionary that can be used to check for process status.
while pulsedmasterlogic.status_dict['sampload_busy']:
    time.sleep(0.1)

{1: 'my_rabi_ch1', 2: 'my_rabi_ch2'}


In [14]:
# You can check on the currently loaded asset on the pulse generator by using the following property:
print('currently loaded asset:', pulsedmasterlogic.loaded_asset)
# You can check on all created waveforms on the pulse generator by using the following property:
print('Available waveforms on device:', pulsedmasterlogic.sampled_waveforms)

currently loaded asset: ('', '')
Available waveforms on device: ['HHtauseq_000_ch1', 'HHtauseq_000_ch2', 'HHtauseq_001_ch1', 'HHtauseq_001_ch2', 'HHtauseq_002_ch1', 'HHtauseq_002_ch2', 'HHtauseq_003_ch1', 'HHtauseq_003_ch2', 'HHtauseq_004_ch1', 'HHtauseq_004_ch2', 'HHtauseq_005_ch1', 'HHtauseq_005_ch2', 'HHtauseq_006_ch1', 'HHtauseq_006_ch2', 'HHtauseq_007_ch1', 'HHtauseq_007_ch2', 'HHtauseq_008_ch1', 'HHtauseq_008_ch2', 'HHtauseq_009_ch1', 'HHtauseq_009_ch2', 'HHtauseq_010_ch1', 'HHtauseq_010_ch2', 'HHtauseq_011_ch1', 'HHtauseq_011_ch2', 'HHtauseq_012_ch1', 'HHtauseq_012_ch2', 'HHtauseq_013_ch1', 'HHtauseq_013_ch2', 'HHtauseq_014_ch1', 'HHtauseq_014_ch2', 'HHtauseq_015_ch1', 'HHtauseq_015_ch2', 'HHtauseq_016_ch1', 'HHtauseq_016_ch2', 'HHtauseq_017_ch1', 'HHtauseq_017_ch2', 'HHtauseq_018_ch1', 'HHtauseq_018_ch2', 'HHtauseq_019_ch1', 'HHtauseq_019_ch2', 'HHtauseq_020_ch1', 'HHtauseq_020_ch2', 'HHtauseq_021_ch1', 'HHtauseq_021_ch2', 'HHtauseq_022_ch1', 'HHtauseq_022_ch2', 'HHtauseq_023_c

# Set up the actual measurement

### Set the measurement settings

In [6]:
# If the pulse sequence has been generated not using predefined methods (e.g. using the graphical table editor) the 
# measurement settings have to be given manually.
# In the same way as setting the generation parameters you can either pass a settings dictionary or single keyword 
# arguments or a combination of both:
pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable=1e-8 + generate_params['number_of_taus'] * 1e-8,
                                           number_of_lasers=generate_params['number_of_taus'], 
                                           laser_ignore_list=[], 
                                           alternating=False, 
                                           units=('s', 'arb. u.'))
time.sleep(0.5)

# If the pulse sequence to run has been generated using predefined methods, the measurement settings can be invoked 
# from the PulseBlockEnsemble attribute "measurement_information". In that case it is enough to execute the following:
# pulsedmasterlogic.set_measurement_settings(invoke_settings=True)
# time.sleep(0.5)

{'number_of_gates': 50}


Traceback (most recent call last):
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\pulsed\pulsed_measurement_logic.py", line 702, in set_measurement_settings
    self._measurement_settings_sanity_check()
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\pulsed\pulsed_measurement_logic.py", line 1029, in _measurement_settings_sanity_check
    if len(self._controlled_variable) < 1:
TypeError: len() of unsized object


In [3]:
# You can get a read-only view of the current measurement settings by using this property:
for setting, value in pulsedmasterlogic.measurement_settings.items():
    print('{0}:\n  {1}'.format(setting, value))

invoke_settings:
  False
controlled_variable:
  1.0099999999999999e-06
number_of_lasers:
  100
laser_ignore_list:
  []
alternating:
  False
units:
  ('s', 'arb. u.')


### Set the fast counter settings (when NOT using "invoke_settings")

In [ ]:
# In case you used "invoke_Settings" this part is not necessary
# The number_of_gates is only used in case of a hardware gated fast counter. It will always be set to 0 otherwise.
# The record length should be the length of the pulse sequence for an ungated counter and the length of the longest 
# gate for a gated counter.
pulsedmasterlogic.set_fast_counter_settings(record_length=237.8e-6, number_of_gates=50)
time.sleep(0.5)

In [17]:
# You can see the current fast counter settings with the this property:
for setting, value in pulsedmasterlogic.fast_counter_settings.items():
    print('{0}: {1}'.format(setting, value))

bin_width: 1e-09
record_length: 3e-06
number_of_gates: 0
is_gated: True


### Set extraction settings

In [28]:
# You can also change how qudi will extract laser pulses from the raw fast counter timetrace.
# You can get all available extraction methods with the following property:
print(list(pulsedmasterlogic.extraction_methods))

['conv_deriv']


In [29]:
# To change the current extraction method:
pulsedmasterlogic.set_extraction_settings(method='threshold')
time.sleep(0.2)

# You can get the current extraction method along with all needed settings with the read-only property:
for setting, value in pulsedmasterlogic.extraction_settings.items():
    print('{0}: {1}'.format(setting, value))

conv_std_dev: 20.0
method: conv_deriv


In [30]:
# Set the extraction settings in the same way as you set other settings
pulsedmasterlogic.set_extraction_settings(count_threshold=20, min_laser_length=100e-9, threshold_tolerance=10e-9)
time.sleep(0.2)

### Set analysis settings

In [31]:
# Completely analogous to the extraction method
print(list(pulsedmasterlogic.analysis_methods))

['mean', 'mean_norm']


In [32]:
# To change the current analysis method:
pulsedmasterlogic.set_analysis_settings(method='mean_norm')
time.sleep(0.2)

# You can get the current analysis method along with all needed settings with the read-only property:
for setting, value in pulsedmasterlogic.analysis_settings.items():
    print('{0}: {1}'.format(setting, value))

signal_start: 0.0
signal_end: 2e-07
norm_start: 3e-07
norm_end: 5e-07
method: mean_norm


In [33]:
# Set the analysis settings in the same way as you set other settings
pulsedmasterlogic.set_analysis_settings(signal_start=0.0, signal_end=2e-7, norm_start=4e-7, norm_end=7e-7)
time.sleep(0.2)

### When using an additional external CW microwave, set microwave settings

In [ ]:
# Set new values
pulsedmasterlogic.set_ext_microwave_settings(power=-20, frequency=2.5e9, use_ext_microwave=True)
time.sleep(0.5)

# get the current settings with the following property:
for setting, value in pulsedmasterlogic.ext_microwave_settings.items():
    print('{0}: {1}'.format(setting, value))

### You can set if a second plot should be shown with a certain data transformation

In [ ]:
pulsedmasterlogic.set_alternative_data_type('FFT')
time.sleep(0.5)
print(pulsedmasterlogic.alternative_data_type)

### Set the refresh rate for the measurement data

In [12]:
# This will set the time interval for reading the raw data from the fast counter and analyzing it. 
# Don't set this interval too small since it can lead to heavy CPU load and crashes.
# If you want to update the data on demand you can always click the "pull and analyze data" button in the GUI.
pulsedmasterlogic.set_timer_interval(3)
time.sleep(0.2)
print(pulsedmasterlogic.timer_interval)

3.0


# Start the pulsed measurement

In [10]:
# If everything is properly set, we can start a measurement simply by calling:
pulsedmasterlogic.toggle_pulsed_measurement(True)
# Wait until the pulsedmeasurementlogic is actually busy and the measurement is running
while pulsedmeasurementlogic.module_state() != 'locked':
    time.sleep(0.2)

# Stop the pulsed measurement

In [11]:
pulsedmasterlogic.toggle_pulsed_measurement(False)
# Wait until the pulsedmeasurementlogic is actually idle and the measurement is stopped
while pulsedmeasurementlogic.module_state() == 'locked':
    time.sleep(0.2)

# Save the measurement data

In [ ]:
pulsedmasterlogic.save_measurement_data(tag='my_dummy_measurement', with_error=True)